# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` は付けない）
* `NOT IN` は NULL 罠のため回避
* 判定は **ID 基準**（主キー `(x,y,z)`）、表示は仕様どおりの列名と順序

## 1) 問題

* `三つの線分 (x, y, z) が三角形を成すか判定し、Yes/No を返す。`
* 入力テーブル例: `Triangle(x INT, y INT, z INT)`（主キー `(x,y,z)`）
* 出力仕様: `x, y, z, triangle`（triangle は `Yes` / `No`）

## 2) 最適解（単一クエリ）

> 三角不等式（任意の二辺の和が残りの一辺より大きい）をそのまま判定。

```sql
SELECT
  x, y, z,
  CASE
    WHEN x + y > z AND x + z > y AND y + z > x THEN 'Yes'
    ELSE 'No'
  END AS triangle
FROM Triangle;

Runtime 259 ms
Beats 81.02%

```

* ウィンドウ関数は不要（行独立判定のため）。
* `NULL` は主キー列のため存在しない前提。
* 0 や負値が入っている場合でも、この条件で自然に `No` になる。

## 3) 代替解

> **辺の並べ替え版**：`least + mid > greatest` に一本化して判定。
> 中間値は「合計 − 最小 − 最大」で求める。

```sql
SELECT
  x, y, z,
  CASE
    WHEN (LEAST(x, y, z) + (x + y + z - LEAST(x, y, z) - GREATEST(x, y, z))) > GREATEST(x, y, z)
      THEN 'Yes'
    ELSE 'No'
  END AS triangle
FROM Triangle;

Runtime 271 ms
Beats 61.68%

```

* 計算回数が一定で、可読性も高い。

## 4) 要点解説

* **方針**: 三角不等式 `a + b > c`, `a + c > b`, `b + c > a`。
  あるいは並べ替えて `a ≤ b ≤ c` とすると **`a + b > c`** のみで十分。
* **NULL / 重複**: 主キーのため `NULL` なし、重複行なし。
* **安定性**: 出力順は任意。`ORDER BY` 非指定で良い。

## 5) 計算量（概算）

* 各行で定数回の算術・比較のみ：**O(N)**。

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Triangle x y z] --> B[三角不等式の判定]
  B -->|Yes| C[triangle = Yes]
  B -->|No| D[triangle = No]
  C --> E[出力 x y z triangle]
  D --> E
```

**堅牢性とちょい微速化**の観点でいくつか手当てできます。

---

## 改善ポイント（実務目線）

1. **1 本の不等式に集約**（並べ替え）

* 三角不等式は `a ≤ b ≤ c` に並べると **`a + b > c` の 1 回だけ**で判定できます。
* 比較回数が 3 → 1 に減るので、わずかに有利。

2. **非正の値を早期除外**

* `x<=0 OR y<=0 OR z<=0` は確実に `No`。先に弾くと分岐の平均コストが下がります（特に不良データ混在時）。

3. **オーバーフロー耐性**

* `INT` どうしの加算は通常 BIGINT に昇格しますが、境界値を意識するなら **明示的に BIGINT 化**しておくと安心。
* もしくは `a + b > c` を **`a > c - b`** に書き換えて加算のオーバーフローを避ける（減算側の境界もあるので BIGINT キャスト併用が無難）。

4. **LEAST/GREATEST の重複計算を避ける**

* 同じ SELECT で複数回呼ぶより、CTE で一度だけ求め、それを再利用。

5. **CASE → IF**（ごく微差）

* MySQL では `IF(cond,'Yes','No')` の方が簡潔で、わずかに軽いことが多いです（誤差レベル）。

---

## 提案クエリ（堅牢・単純・軽量）

```sql
WITH base AS (
  SELECT
    x, y, z,
    LEAST(x, y, z) AS a,
    GREATEST(x, y, z) AS c
  FROM Triangle
),
ext AS (
  SELECT
    x, y, z, a, c,
    (x + y + z) - a - c AS b  -- 中間値
  FROM base
)
SELECT
  x, y, z,
  IF(
    x > 0 AND y > 0 AND z > 0
    AND CAST(a AS SIGNED) + CAST(b AS SIGNED) > CAST(c AS SIGNED),
    'Yes', 'No'
  ) AS triangle
FROM ext;

Runtime 265 ms
Beats 71.17%

```

### 変形（オーバーフロー回避をより厳密にしたい場合）

```sql
WITH base AS (
  SELECT
    x, y, z,
    LEAST(x, y, z) AS a,
    GREATEST(x, y, z) AS c
  FROM Triangle
),
ext AS (
  SELECT
    x, y, z, a, c,
    (x + y + z) - a - c AS b
  FROM base
)
SELECT
  x, y, z,
  IF(
    x > 0 AND y > 0 AND z > 0
    AND CAST(a AS SIGNED) > CAST(c AS SIGNED) - CAST(b AS SIGNED),
    'Yes', 'No'
  ) AS triangle
FROM ext;

Runtime 256 ms
Beats 85.53%

```

* いずれも **LEAST/GREATEST は 1 回ずつ**のみ評価。
* 判定は **1 比較**（早期除外を除く）。
* CAST は境界値対策（通常データならほぼ誤差コスト）。

---

## どれくらい速くなる？

* 行あたりの演算回数削減（3 比較 → 1 比較、関数の再計算回避）で **微改善**が期待できますが、259 ms というオーダーなら **I/O や実行環境のノイズ**が支配的になることも多いです。
* つまり「必ず劇的に速くなる」類ではなく、**堅牢性（境界値）＋わずかな算術削減**が主目的だと捉えてください。

---

## 実務でさらに踏み込むなら

* 高頻度で同判定を使うなら、**仮想生成列**（`a`, `b`, `c`, あるいは `two_small_sum = a+b`）を用意してビューに載せておくと再利用性と可読性が上がります（LeetCode では不可）。
* データ品質が担保されている（正の整数のみ）なら、`x>0…` の早期除外は省略可。

---

結論：現在のクエリでも十分合格点ですが、上の **「並べ替え＋1 比較」版**にすると、**堅牢で読みやすく、ほんのり速い**実装になります。

